In [1]:
import pandas as pd

experimentd_df = pd.read_csv("experiments.tsv", delimiter="\t")
experimentd_df.head()

,exp_hier,vista_id,allele_id,backbone,stage,curation_status,description,denominator,tissue,tissue_positive,...,coord,strand,coordinate_hg38,coordinate_mm10,bracket_hg38,bracket_mm10,seq_hg38,seq_mm10,seq,external_note
0,03vn00010001,hs1,0,hZR,e11.5,positive,reference,15.0,lb;hb;nt;cn,3;12;12;8,...,chr16:86396481-86397120,+,chr16:86396481-86397120,chr8:121002895-121003512,IRF8-FOXF1,Gm20388 (intragenic),AACTGAAGGGACCCCGTTAGCATATAAACAAAAGGTGGGGGGTAGC...,AGTTGAGCGACCCTGTTAACGTATAAACAAAAGGTGGGGGGTAACC...,AACTGAAGGGACCCCGTTAGCATATAAACAAAAGGTGGGGGGTAGC...,NaN
1,03v300010001,hs2,0,hZR,e11.5,negative,reference,NaN,NaN,NaN,...,chr16:85586489-85588130,+,chr16:85586489-85588130,chr8:120516455-120517853,GSE1 (intragenic),Gm20388 (intragenic),GGCCCTGGTATGTTTGTTCTTCCAGGGGCTCCCAGGATGGATCCAG...,ATATTGGCTCCTACAGGGGCTCCAGGATCATTCCCTGCCCTCCTGG...,GGCCCTGGTATGTTTGTTCTTCCAGGGGCTCCCAGGATGGATCCAG...,NaN
2,000100010001,hs3,0,hZR,e11.5,negative,reference,NaN,NaN,NaN,...,chr16:80389446-80390755,+,chr16:80389446-80390755,chr8:116384137-116385431,MAF-DYNLRB2,Maf-Dynlrb2,AAGATTGCCATTTGGGGTGTTTCTTGGGGCTAAGAACCATGAAGAC...,GAGATTGCCATTTGGGGTGTGTCTTGGGGGCTAAGTGCCATGAAGA...,AAGATTGCCATTTGGGGTGTTTCTTGGGGCTAAGAACCATGAAGAC...,NaN
3,000200010001,hs4,0,hZR,e11.5,positive,reference,10.0,mb;hb;nt,10;10;6,...,chr16:80338700-80339858,+,chr16:80338700-80339858,chr8:116335161-116337629,MAF-DYNLRB2,Maf-Dynlrb2,CAGAGACAGACAGTGACAGAGACAGATTTTAGAATTTGAACAAAGG...,catgtgtgtatgtgtgcatatgtgtgtgtgtgCTGGCGCACATTGA...,CAGAGACAGACAGTGACAGAGACAGATTTTAGAATTTGAACAAAGG...,NaN
4,03ya00010001,hs5,0,hZR,e11.5,negative,reference,NaN,NaN,NaN,...,chr16:79936010-79937400,+,chr16:79936010-79937400,chr8:116013572-116015153,MAF-DYNLRB2,Maf-Dynlrb2,TGACACCCACTATTATCCAGTCCTTGATAAACCTCTTTATTTGTTC...,TGTCTCCTACAGCTATAGTTCTTCAGTTATTGACAGAACAGACCCT...,TGACACCCACTATTATCCAGTCCTTGATAAACCTCTTTATTTGTTC...,NaN


In [15]:
positive_df = experimentd_df.loc[
    experimentd_df["curation_status"] == "positive", ["seq_hg38"]
]
positive_df = positive_df[positive_df["seq_hg38"].notna()]
positive_df["class"] = 1
print(positive_df.shape[0])

2267


In [16]:

negative_df = experimentd_df.loc[experimentd_df['curation_status'] == 'negative', ['seq_hg38']]
negative_df = negative_df[negative_df['seq_hg38'].notna()]
negative_df['class'] = 0
print(negative_df.shape[0])

1913


## Preprocessing

In [4]:
from Bio.Seq import Seq
from itertools import product


def count_kmers(seqence, k):
    nucleotides = ["A", "C", "G", "T"]
    all_subsequences = ["".join(seq) for seq in product(nucleotides, repeat=k)]
    k_mers = set([])
    for k_mer in all_subsequences:
        seq = Seq(k_mer)
        if str(seq.reverse_complement()) not in k_mers:
            k_mers.add(seq)

    seq = Seq(seqence.upper())
    k_mer_freq = {
        key: (
            seq.count_overlap(key)
            + (
                seq.count_overlap(key.reverse_complement())
                if key != key.reverse_complement()
                else 0
            )
        )
        / len(seq)
        for key in k_mers
    }
    return [_freq for _seq, _freq in sorted(k_mer_freq.items())]

In [5]:
def preprocess_data(src_df, k):
    df = pd.DataFrame()
    df["class"] = src_df["class"]
    features_len = len(count_kmers("_", k))
    features_df = src_df["seq_hg38"].apply(lambda x: pd.Series(count_kmers(x, k)))
    features_df.columns = ["x" + str(i) for i in range(features_len)]

    df = pd.concat([df, features_df], axis=1)
    X = df.drop(columns=["class"])
    Y = df.loc[:, "class"]
    return X, Y

## Training

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import (
    accuracy_score,
    recall_score,
    precision_score,
    roc_auc_score,
    f1_score,
    confusion_matrix,
)


def evaluate_model(y_pred, y_proba, y_true):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_proba)
    f1 = f1_score(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred)

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "roc_auc": roc_auc,
        "f1_score": f1,
    }, conf_matrix


def train_and_evaluate_model(pos_df, neg_df, k):
    train_pos, test_pos = train_test_split(pos_df, test_size=0.3, random_state=0)
    train_neg, test_neg = train_test_split(neg_df, test_size=0.3, random_state=0)
    X_train, Y_train = preprocess_data(pd.concat([train_pos, train_neg], axis=0), k)
    X_test, Y_test = preprocess_data(pd.concat([test_pos, test_neg], axis=0), k)

    model = LogisticRegressionCV(cv=10, random_state=0, n_jobs=-1)
    model.fit(X_train, Y_train)
    cv_mean = model.scores_[1].mean(axis=0)
    cv_std = model.scores_[1].std(axis=0)
    max_cv_mean_i = cv_mean.argmax()
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]
    metrics, matrices = evaluate_model(y_pred, y_proba, Y_test)
    metrics["cv_accuracy"] = cv_mean[max_cv_mean_i]
    metrics["cv_accuracy_std"] = cv_std[max_cv_mean_i]
    return metrics, matrices

## Evaluation

In [7]:
scores_list = []
conf_matrices = []
for k in [3, 4, 5]:
    metrics, conf_matrix = train_and_evaluate_model(positive_df, negative_df, k)
    metrics["k"] = k
    scores_list.append(metrics)
    conf_matrices.append(conf_matrix)
scores_df = pd.DataFrame(scores_list)
scores_df.head()

,accuracy,precision,recall,roc_auc,f1_score,cv_accuracy,cv_accuracy_std,k
0,0.564940,0.576792,0.744493,0.584744,0.650000,0.570596,0.023293,3
1,0.576096,0.585338,0.750367,0.587717,0.657658,0.582572,0.027811,4
2,0.556972,0.555457,0.919236,0.579126,0.692478,0.564098,0.010428,5


In [22]:
conf_matrices[1]

array([[212, 362],
       [170, 511]])

## Random negative data from GRCh38

In [9]:
import random
from Bio import SeqIO

sample_sequences = {} 
for record in SeqIO.parse('GRCh38.p14.genome.fa', format='fasta'):
    sample_sequences[record.id] = record.seq
    if len(sample_sequences) == 23: break

seq_lengths = positive_df['seq_hg38'].apply(len).tolist()
random.seed(0)
positive_sequences = set(positive_df['seq_hg38'].apply(str.upper).tolist())

positive_coordinates_df = experimentd_df.loc[experimentd_df['curation_status'] == 'positive', ['coordinate_hg38']]
positive_coordinates_df = positive_coordinates_df[positive_coordinates_df['coordinate_hg38'].notna()]
positive_coordinates_df['chromosome'] = positive_coordinates_df['coordinate_hg38'].apply(lambda x: x.split(':')[0])
positive_coordinates_df['range'] = positive_coordinates_df['coordinate_hg38'].apply(lambda x: [int(item) for item in x.split(':')[1].split('-')])
positive_coordinastes = positive_coordinates_df.groupby('chromosome')['range'].apply(list).to_dict()

negative_sequences = []
for length in seq_lengths:
    sample_chromosome = random.choice(list(sample_sequences.keys()))
    sample_sequence = sample_sequences[sample_chromosome]
    coordinates_to_exclude = positive_coordinastes[sample_chromosome]
    start = random.randint(0,len(sample_sequence) - length)
    end_excl = start + length
    sequence = sample_sequence[start:end_excl]
    while sequence.count("N") > 0 or any(
        [
            start >= coord[0] and start <= coord[1] or end_excl - 1 >= coord[0] and end_excl -1 <= coord[1]
            for coord in coordinates_to_exclude
        ]
    ):
        start = random.randint(0, len(sample_sequence) - length)
        end_excl = start + length
        sequence = sample_sequence[start:end_excl]
    negative_sequences.append(str(sequence).upper())

In [10]:
artificial_negative_df = pd.DataFrame()
artificial_negative_df["seq_hg38"] = negative_sequences
artificial_negative_df["class"] = 0

artificial_positive_df = pd.DataFrame()
artificial_positive_df["seq_hg38"] = positive_df["seq_hg38"]
artificial_positive_df["class"] = 1

### Evaluation

In [11]:
scores_list_a = []
conf_matrices_a = []
for k in [3, 4, 5]:
    metrics, conf_matrix = train_and_evaluate_model(
        artificial_positive_df, artificial_negative_df, k
    )
    metrics["k"] = k
    scores_list_a.append(metrics)
    conf_matrices_a.append(conf_matrix)
scores_df_a = pd.DataFrame(scores_list_a)
scores_df_a.head()

,accuracy,precision,recall,roc_auc,f1_score,cv_accuracy,cv_accuracy_std,k
0,0.751101,0.752212,0.748899,0.815483,0.750552,0.747168,0.018725,3
1,0.779001,0.790520,0.759178,0.844424,0.774532,0.776164,0.020641,4
2,0.792217,0.794379,0.788546,0.854701,0.791452,0.780583,0.022453,5


In [20]:
conf_matrices_a[2]

array([[542, 139],
       [144, 537]])

### Evaluate model trained with artificial negative on vista negative data

In [13]:
from sklearn.metrics import confusion_matrix

k = 4
train_pos, test_pos = train_test_split(
    artificial_positive_df, test_size=0.3, random_state=0
)
train_neg, _ = train_test_split(
    artificial_negative_df, test_size=0.3, random_state=0
)
_, test_neg = train_test_split(
    negative_df, test_size=0.3, random_state=0
)
X_train, Y_train = preprocess_data(pd.concat([train_pos, train_neg], axis=0), k)


print(f'Size of test data: positive {test_pos.shape[0]}, negative {test_neg.shape[0]}')
model = LogisticRegressionCV(cv=10, random_state=0, n_jobs=-1)
model.fit(X_train, Y_train)
X_test, Y_test = preprocess_data(pd.concat([test_neg, test_pos], axis=0), k)
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]
metrics, conf_matrix = evaluate_model(y_pred, y_proba, Y_test)
a_eval_df = pd.DataFrame([metrics])
print(conf_matrix)
a_eval_df.head()

Size of test data: positive 681, negative 574
[[182 392]
 [164 517]]


,accuracy,precision,recall,roc_auc,f1_score
0,0.556972,0.568757,0.759178,0.554744,0.650314


### Evaluate model trained with vista data on artificial negative data

In [14]:
from sklearn.metrics import confusion_matrix

k = 4
train_pos, test_pos = train_test_split(positive_df, test_size=0.3, random_state=0)
train_neg, _ = train_test_split(negative_df, test_size=0.3, random_state=0)
_, test_neg = train_test_split(artificial_negative_df, test_size=0.3, random_state=0)
X_train, Y_train = preprocess_data(pd.concat([train_pos, train_neg], axis=0), k)
print(f'Size of test data: positive {test_pos.shape[0]}, negative {test_neg.shape[0]}')

model = LogisticRegressionCV(cv=10, random_state=0, n_jobs=-1)
model.fit(X_train, Y_train)
X_test, Y_test = preprocess_data(
    pd.concat([test_pos, test_neg], axis=0), k
)
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]
metrics, conf_matrix = evaluate_model(y_pred, y_proba, Y_test)
a2_eval_df = pd.DataFrame([metrics]).head()
print(conf_matrix)
a2_eval_df.head()

Size of test data: positive 681, negative 681
[[424 257]
 [170 511]]


,accuracy,precision,recall,roc_auc,f1_score
0,0.68649,0.665365,0.750367,0.746939,0.705314
